# I. Введение логарифмического барьера, выпуклость задачи.

## 1. Критерий выпуклости второго порядка

Optimal allocation with resource constraint:
$$f_{i}\left(x_{i}\right) = a_{i}e^{x_{i}}, a_{i} > 0, i \in \{ 1, ..., n\}$$

Также ограничение бюджета:
$$\sum^{n}_{i=1}x_{i} = b, b \in R$$

Дополнительное ограничение:
$$x_i \geq \frac{b}{2n} $$

Задача минимизации:
$$\sum^{n}_{i=1}a_{i}e^{x_{i}} \rightarrow min, a_{i} > 0, i \in \{ 1, ..., n\}$$

Задача с ограничениями:
$$\sum^{n}_{i=1}a_{i}e^{x_{i}} \rightarrow min, a_{i} > 0, i \in \{ 1, ..., n\},$$
$$
S.t. \sum^{n}_{i=1}x_{i} = b, \

x_i \geq \frac{b}{2n}, i \in \{ 1, ..., n\},  \in R$$

Обозначим ограничение-неравенство через $g(x_i) = \frac{b}{2n} - x_i \leq 0$.

Введем логарифмический барьер:
$$
\phi(x) = -\frac{1}{t} \sum_{i=1}^{n} \ln(-g(x_i)) = -\frac{1}{t} \sum_{i=1}^{n} \ln( x_i - \frac{b}{2n}), где   t > 0 - фикс.
$$ 

Итоговая задача с барьером:

$$\sum^{n}_{i=1} \left( a_{i} e^{x_{i}} -\frac{1}{t} \ln( x_i - \frac{b}{2n}) \right) \rightarrow min, a_{i} > 0, i \in \{ 1, ..., n\}, t > 0$$
$$
S.t. \sum^{n}_{i=1}x_{i} = b, b  \in R$$

Для изучения выпуклости рассмотрим гессиан оптимизируемой функции:
$$H(x)_{i,j} = 0, i \neq j$$
$$H(x)_{i,j} = a_{i}e^{x_{i}} + \frac{1}{t(x_i - \frac{b}{2n})^2}, i = j$$
$$\text{При } a_{i} > 0 \text{ и } t > 0 \text{ очевидно, что } H(x)_{i,i} > 0, \text{ так как } \forall x \in R: e^{x_{i}} > 0 \text{ и } \frac{1}{(x_i - \frac{b}{2n})^2} > 0$$
Матрица такого вида является положительно определенной.

Так же рассмотрим знакоопределенность гессиана ограничения:
$$H(x) = \textbf{0}$$ 

Таким образом, рассматриваемая барьерная задача является выпуклой.

# II. Решение тестовых примеров

In [1]:
import numpy as np
import cvxpy as cp
import time
import polars as pl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 1. Генерация примеров

In [25]:
n_dims = range(10, 110, 10)
n_examples = 5
n_seeds = 5

In [28]:
raw_experiment_sheet = pl.DataFrame({"dim": n_dims}).select(
    pl.all().repeat_by(n_examples).flatten()
)

In [29]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    pl.col("dim")
    .map_elements(
        lambda dim_: np.random.uniform(low=0.1, high=10, size=dim_),
        return_dtype=pl.List(float),
        returns_scalar=False,
    )
    .alias("a"),
    pl.col("dim")
    .map_elements(
        lambda dim_: np.random.uniform(low=0, high=10, size=1),
        return_dtype=float,
        returns_scalar=True,
    )
    .alias("b"),
)

In [30]:
raw_experiment_sheet

dim,a,b
i64,list[f64],f64
10,"[2.266699, 6.338006, … 9.679609]",0.536932
10,"[1.494705, 2.642139, … 0.857276]",5.722819
10,"[0.705691, 4.232732, … 9.117765]",0.943189
10,"[3.910042, 2.236563, … 6.208344]",5.714863
10,"[2.658745, 5.041897, … 5.856035]",4.240225
…,…,…
100,"[7.183344, 8.056831, … 2.8046]",3.50619
100,"[3.958926, 2.361616, … 2.60997]",5.235832
100,"[2.901908, 4.285501, … 8.568109]",3.268357


## 2. Решение солвером CVXPY

In [31]:
def solve_oawrc(a, b):

    a = np.asarray(a)
    n = a.shape[0]
    
    x = cp.Variable(n)

    objective = cp.Minimize(cp.sum(cp.multiply(a, cp.exp(x))))

    constraints = [cp.sum(x) - b == 0, x >= b / (2 * n)]

    prob = cp.Problem(objective, constraints)

    try:
        start_t = time.time()
        result = prob.solve()
        sovle_time = time.time() - start_t
    except:
        res = {"cvxpy_x": None, "cvxpy_y": None, "cvxpy_t": None}
    else:
        res = {"cvxpy_x": list(x.value), "cvxpy_y": result, "cvxpy_t": sovle_time}

    return res

In [32]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    (
        pl.struct("a", "b")
        .map_elements(
            lambda x: solve_oawrc(**x),
            return_dtype=pl.Struct(
                {"cvxpy_x": pl.List(float), "cvxpy_y": float, "cvxpy_t": float}
            ),
        )
        .alias("cvxpy_out")
    )
).unnest("cvxpy_out")

In [33]:
raw_experiment_sheet

dim,a,b,cvxpy_x,cvxpy_y,cvxpy_t
i64,list[f64],f64,list[f64],f64,f64
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559
10,"[1.494705, 2.642139, … 0.857276]",5.722819,"[0.967273, 0.39774, … 1.523446]",56.113635,0.023103
10,"[0.705691, 4.232732, … 9.117765]",0.943189,"[0.04716, 0.047159, … 0.047159]",40.029509,0.016991
10,"[3.910042, 2.236563, … 6.208344]",5.714863,"[0.285743, 0.285743, … 0.285743]",50.715963,0.012339
10,"[2.658745, 5.041897, … 5.856035]",4.240225,"[0.276565, 0.212011, … 0.212011]",64.778584,0.008978
…,…,…,…,…,…
100,"[7.183344, 8.056831, … 2.8046]",3.50619,"[0.017531, 0.017531, … 0.017531]",499.712984,0.010377
100,"[3.958926, 2.361616, … 2.60997]",5.235832,"[0.026179, 0.026179, … 0.026179]",500.594255,0.011275
100,"[2.901908, 4.285501, … 8.568109]",3.268357,"[0.016342, 0.016342, … 0.016342]",518.123097,0.010482


## 3. Решение прямой задачи методом Ньютона 

In [34]:
def random_sum_bound(dim, b):
    rnd = np.random.uniform(size=dim)
    y = rnd / rnd.sum() * (b / 2)
    return list(y + b / (2 * dim))

In [35]:
raw_experiment_sheet = raw_experiment_sheet.select(
    pl.all().repeat_by(n_seeds).flatten()
).with_columns(
    pl.struct("dim", "b")
    .map_elements(lambda x: random_sum_bound(**x), return_dtype=pl.List(float))
    .alias("x_init")
)

In [36]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
        pl.lit([2, 10, 50, 100]).alias("mu")
    ).explode("mu")                      

raw_experiment_sheet = raw_experiment_sheet.drop_nans()
raw_experiment_sheet

dim,a,b,cvxpy_x,cvxpy_y,cvxpy_t,x_init,mu
i64,list[f64],f64,list[f64],f64,f64,list[f64],i64
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.062119, 0.075646, … 0.058166]",2
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.062119, 0.075646, … 0.058166]",10
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.062119, 0.075646, … 0.058166]",50
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.062119, 0.075646, … 0.058166]",100
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.067978, 0.03281, … 0.069219]",2
…,…,…,…,…,…,…,…
100,"[6.185764, 5.141971, … 7.544346]",7.533661,"[0.037668, 0.037668, … 0.037668]",547.174836,0.00946,"[0.106166, 0.053893, … 0.106945]",100
100,"[6.185764, 5.141971, … 7.544346]",7.533661,"[0.037668, 0.037668, … 0.037668]",547.174836,0.00946,"[0.083529, 0.084167, … 0.091702]",2
100,"[6.185764, 5.141971, … 7.544346]",7.533661,"[0.037668, 0.037668, … 0.037668]",547.174836,0.00946,"[0.083529, 0.084167, … 0.091702]",10


In [37]:
def f(x, a):
    return np.dot(a, np.exp(x)).reshape(1,1)

def grad_f(x, a):
    return (a * np.exp(x)).reshape(1,-1)

def hessian_f(x, a):
    return np.diag(a * np.exp(x))

def barrier_f(x, a, b, t):
    return (t * np.dot(a, np.exp(x)) - np.sum(np.log(x - b / (2*len(a))))).reshape(1,1)

def barrier_grad_f(x, a, b, t):
    return (t * a * np.exp(x) - 1. / (x - b/(2*len(a)))).reshape(1,-1)

def barrier_hessian_f(x, a, b, t):
    return np.diag(t * a * np.exp(x) + 1. / (x - b / (2*len(a)))**2 )

def g(x, b):
    return np.array([np.sum(np.asarray(x)) - b]).reshape(1,1)

def grad_g(x):
    return np.ones(len(x)).reshape(1,-1)

def hessian_g(x):
    return np.zeros([len(x)]*2)

In [ ]:
def armijo_line_search(x, a, b, t, dx, alpha0=1.0, beta=0.5, c=1e-4, max_iters=20):
    alpha = alpha0

    func_value = barrier_f(x, a, b, t)[0][0]
    gTd  = np.dot(barrier_grad_f(x, a, b, t), dx)
    
    for _ in range(max_iters):
        x_new = x + alpha * dx
        func_value_new = barrier_f(x_new, a, b, t)[0][0]

        if np.all(x_new >= b / (2*len(a))) and np.allclose(np.sum(x_new), b) and func_value_new < func_value + c * alpha * gTd:
            return alpha
        alpha *= beta
    
    return None

def solve_oawrc_newton(a, b, x_init, cvxpy_y, t, eps=0.01, max_iters=20, max_linesearch_iters=30, c=0.2, beta=0.5):

    x = np.asarray(x_init)
    a = np.asarray(a)
    
    for i in range(max_iters):
        grad_L = barrier_grad_f(x, a, b, t)
        hessian_L = barrier_hessian_f(x, a, b, t)

        hessian_L_full = np.block([
            [hessian_L, grad_g(x).T],
            [grad_g(x), np.zeros([1,1])]
        ])
        
        grad_L_full = np.concatenate([grad_L, g(x, b).reshape(1,-1)], axis = -1)
        d = np.linalg.solve(hessian_L_full, -grad_L_full.T)
        dx = d[:len(x),0]

        alpha = armijo_line_search(x, a, b, t, dx, alpha0=1.0, beta=beta, c=c, max_iters=max_linesearch_iters)
        if alpha is None:
            break

        x = x + alpha * dx

        curr_f = f(x, a)[0][0]
        if np.abs(curr_f - cvxpy_y) < eps:
            break

    return x.tolist(), i+1, curr_f

def barrier_method(a, b, x_init, cvxpy_y, mu=2, t_init=1, eps=0.001, max_centering_iters=20):
    a = np.asarray(a)
    n = a.shape[0]
    x = x_init.copy()
    t = t_init

    sovle_time = time.time()
    newton_iters_hist, newton_hist_y = [], []

    while n/t >= eps:
        x, centering_iters, curr_y = solve_oawrc_newton(a, b, x, cvxpy_y, t, eps=eps, max_iters=max_centering_iters)

        newton_iters_hist.append(centering_iters)
        newton_hist_y.append(curr_y)

        t *= mu

    return {"newton_x": x, "newton_y": curr_y, "newton_t": time.time() - sovle_time, "newton_hist_y": newton_hist_y, "newton_iters_hist": newton_iters_hist}

In [ ]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    (
        pl.struct("a", "b", "x_init", "cvxpy_y", "mu")
        .map_elements(
            lambda x: barrier_method(**x),
            return_dtype=pl.Struct(
                {"newton_x": pl.List(float), "newton_y": float, "newton_t": float, "newton_hist_y": pl.List(float), "newton_iters_hist": pl.List(int)}
            ),
        )
        .alias("newton_out")
    )
).unnest("newton_out")

In [42]:
raw_experiment_sheet

dim,a,b,cvxpy_x,cvxpy_y,cvxpy_t,x_init,mu,newton_x,newton_y,newton_t,newton_hist_y,newton_iters_hist
i64,list[f64],f64,list[f64],f64,f64,list[f64],i64,list[f64],f64,f64,list[f64],list[i64]
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.062119, 0.075646, … 0.058166]",2,"[0.026935, 0.026869, … 0.02686]",48.286135,0.123808,"[49.286848, 49.221879, … 48.286135]","[9, 4, … 1]"
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.062119, 0.075646, … 0.058166]",10,"[0.026889, 0.026859, … 0.026854]",48.285678,0.027595,"[49.286848, 48.845302, … 48.285678]","[9, 7, … 2]"
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.062119, 0.075646, … 0.058166]",50,"[0.027135, 0.026918, … 0.026891]",48.288356,0.022317,"[49.286848, 48.436636, 48.288356]","[9, 9, 12]"
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.062119, 0.075646, … 0.058166]",100,"[0.027022, 0.026854, … 0.026852]",48.285772,0.01519,"[49.286848, 48.363123, 48.285772]","[9, 9, 4]"
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.067978, 0.03281, … 0.069219]",2,"[0.026935, 0.026869, … 0.02686]",48.286135,0.068592,"[49.286848, 49.221879, … 48.286135]","[9, 4, … 1]"
…,…,…,…,…,…,…,…,…,…,…,…,…
100,"[6.185764, 5.141971, … 7.544346]",7.533661,"[0.037668, 0.037668, … 0.037668]",547.174836,0.00946,"[0.106166, 0.053893, … 0.106945]",100,"[0.037687, 0.037691, … 0.037683]",547.184134,0.027852,"[564.188621, 548.079427, 547.184134]","[11, 17, 17]"
100,"[6.185764, 5.141971, … 7.544346]",7.533661,"[0.037668, 0.037668, … 0.037668]",547.174836,0.00946,"[0.083529, 0.084167, … 0.091702]",2,"[0.037671, 0.037672, … 0.037671]",547.176256,0.091363,"[564.188621, 562.7755, … 547.176256]","[12, 5, … 4]"
100,"[6.185764, 5.141971, … 7.544346]",7.533661,"[0.037668, 0.037668, … 0.037668]",547.174836,0.00946,"[0.083529, 0.084167, … 0.091702]",10,"[0.03767, 0.03767, … 0.037669]",547.175453,0.035969,"[564.188621, 554.939015, … 547.175453]","[12, 8, … 2]"


# III. Анализ зависимостей

In [47]:
res_data = raw_experiment_sheet.clone()
res_data = (res_data
    .with_columns(pl.col("newton_hist_y").list.len().alias("iter_num"))
    .with_columns((pl.col("cvxpy_y") - pl.col("newton_y")).abs().alias("y_diff"))
    .select("dim", "mu", "newton_t", "iter_num", "y_diff")
    .group_by("dim", "mu")
    .agg(pl.all().mean().name.suffix("_mean"))
    .sort("dim", "mu")
)

# IV. Выводы